In [1]:
#Tratamiento de datos
# ==============================================================================
import string #encontrar texto no ASCII
import re #Biblioteca para expresiones regulares
import pandas as pd
import numpy as np
# import unicodedata 
from unicodedata import normalize

#
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


C:\Users\Toshiba\Documents\utpl_empresas\envoracle\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [80]:
# importing module 
import cx_Oracle 


# Create a table in Oracle database 
try: 
    sql_conn = cx_Oracle.connect('system/L2022utpl*@localhost:1521/orcl') 

    # Now execute the sqlquery 
    cursor = sql_conn.cursor() 
    query_sql1 = '''select * from PRUEBA'''
    query_sql2 = '''select * from NER_PRUEBA'''
    df1 = pd.read_sql(query_sql1, sql_conn)
    df2 = pd.read_sql(query_sql2, sql_conn)
    
    print("Successful Connection") 
    

except cx_Oracle.DatabaseError as e: 
    print("There is a problem with Oracle", e) 

# by writing finally if any error occurs 
# then also we can close the all database operation 

C:\Users\Toshiba\Documents\utpl_empresas\envoracle\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Successful Connection


C:\Users\Toshiba\Documents\utpl_empresas\envoracle\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [95]:
def clearempresas (columna):
    #Eliminar caracteres especiales
#     (re.sub(r"W+|_", " ", string))
    columna = re.sub("\?|\'|\!|\"|\_|\#|\<br>|\@|\.|\<Br>|\(|\)|\-|\,|\/|\´|\:|\¨"," ",str(columna))
    #Eliminar tildes
    columna = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",normalize( "NFD", columna), 0, re.I)
    # -> NFC
    columna = normalize( 'NFC', columna)
    columna = re.sub(' +', ' ', columna.strip())
    columna = columna.title()
    
    #Eliminar espacios duplicados
    #columna.replace(" ", "")|\¨
    #columna = re.sub("\?|\'|\!|\“|\”|\"|\_|\#|\<br>|\@|\.|\"|\<Br>|\(|\)|\-|\,|\/|\´|\¨|\:|\”"," ",str(columna)

    return  columna


In [96]:

def replace_names(cadena):
    #Encuetre al inicio de cadena las palabras GAD|municipio|
    replace = re.compile(r'\b(GAD|municipio)\b')
    new_list = replace.sub("GOBIERNO AUTONOMO DESCENTRALIZADO", str(cadena))
    return new_list



In [97]:
df2['NER'] = df2['NER'].apply(clearempresas)
# df1['EMPRESA'] = df1['EMPRESA'].apply(replace_names)

In [104]:
df2

,NER,ID
0,Ediloja Cia Ltda,0
1,Lojasoft Solutions,1
2,Cooperativa De Ahorro Y Credito Jardin Azuayo ...,2
3,Universidad Tecnica Particular De Loja,3
4,Universidad Tecnica Particular De Loja,4
5,Mepalecuador S A,5
6,Lizbeth Carolina Pacheco Guevara,6
7,Universidad Tecnica Particular De Loja,7
8,Academy Support Center Universidad San Francis...,8


In [99]:
import es_core_news_sm
import spacy
#loads the model as snlp
nlp = spacy.load("es_core_news_sm")


In [100]:
empresas_ner = df2['NER'].to_string()
d = nlp(empresas_ner)
# document = nlp(empresas_ner)
# len(document)
# print(empresas_ner)
# empresas_ner.ents
# for named_entity in document.ents:
#     ent = named_entity.label_
#     print(named_entity,ent)

In [101]:
empresas_ner

'0                                     Ediloja Cia Ltda\n1                                   Lojasoft Solutions\n2    Cooperativa De Ahorro Y Credito Jardin Azuayo ...\n3               Universidad Tecnica Particular De Loja\n4               Universidad Tecnica Particular De Loja\n5                                     Mepalecuador S A\n6                     Lizbeth Carolina Pacheco Guevara\n7               Universidad Tecnica Particular De Loja\n8    Academy Support Center Universidad San Francis...'

In [102]:
from spacy import displacy
# print named entities in article
print([(word, word.ent_type_) for word in d if word.ent_type_])

# visualize named entities
displacy.render(d, style='ent', jupyter=True)

[(Cia, 'ORG'), (Cooperativa, 'MISC'), (De, 'MISC'), (Ahorro, 'MISC'), (Credito, 'PER'), (Jardin, 'PER'), (Azuayo, 'PER'), (Universidad, 'ORG'), (Tecnica, 'ORG'), (Particular, 'ORG'), (De, 'ORG'), (Universidad, 'ORG'), (Tecnica, 'ORG'), (Particular, 'ORG'), (De, 'ORG'), (Carolina, 'PER'), (Pacheco, 'PER'), (Universidad, 'ORG'), (Tecnica, 'ORG'), (Particular, 'ORG'), (De, 'ORG'), (Academy, 'LOC'), (Support, 'LOC'), (Center, 'LOC'), (San, 'LOC'), (Francis, 'LOC'), (..., 'LOC')]


In [93]:
named_entities = []
for sentence in d:
    temp_entity_name = ''
    temp_named_entity = None
#     sentence = nlp(sentence)
    for word in d:
        term = word.text 
        tag = word.ent_type_
        if tag:
            temp_entity_name = ' '.join([temp_entity_name, term]).strip()
            temp_named_entity = (temp_entity_name, tag)
        else:
            if temp_named_entity:
                named_entities.append(temp_named_entity)
                temp_entity_name = ''
                temp_named_entity = None

entity_frame = pd.DataFrame(named_entities, 
                            columns=['Entity Name', 'Entity Type'])


In [94]:
entity_frame

,Entity Name,Entity Type
0,cia,ORG
1,CREDITO JARDIN AZUAYO,MISC
2,UNIVERSIDAD TECNICA PARTICULAR,ORG
3,UNIVERSIDAD TECNICA PARTICULAR,ORG
4,CAROLINA,ORG
...,...,...
464,UNIVERSIDAD TECNICA PARTICULAR,ORG
465,UNIVERSIDAD TECNICA PARTICULAR,ORG
466,CAROLINA,ORG
467,UNIVERSIDAD TECNICA PARTICULAR,ORG


In [70]:
entity_frame.to_csv('ner.csv')

In [87]:
# get the top named entities
top_entities = (entity_frame.groupby(by=['Entity Name', 'Entity Type'])
                           .size()
                           .sort_values(ascending=False)
                           .reset_index().rename(columns={0 : 'Frequency'}))
top_entities.T.iloc[:,:10]

,0,1,2,3,4
Entity Name,UNIVERSIDAD TECNICA PARTICULAR,ACADEMY SUPPORT,CAROLINA,CREDITO JARDIN AZUAYO,cia
Entity Type,ORG,MISC,ORG,MISC,ORG
Frequency,201,67,67,67,67
